# History and Data Analysis Collaboration System

This notebook implements a multi-agent collaboration system that combines historical research with data analysis to answer complex historical questions. It leverages the power of large language models to simulate specialized agents working together to provide comprehensive answers.

## Key Components
1. **Agent Class**
2. **History Research Agent**
3. **Data Analysis Agent**
4. **History Data Collaboration System**

In [5]:
import os
import time

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from typing import List, Dict
from dotenv import load_dotenv

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')

/Users/tetro/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/tetro/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

### Create Agent class

In [7]:
class Agent:
    def __init__(self, name: str, role: str, skills: List[str]):
        self.name = name
        self.role = role
        self.skills = skills
        self.llm = llm

    def process(self, task: str, context: List[Dict] = None) -> str:
        messages = [
            SystemMessage(content=f"You are {self.name}, a {self.role}. Your skills include: {', '.join(self.skills)}. Respond to the task based on your role and skills.")
        ]
        
        if context:
            for msg in context:
                if msg['role'] == 'human':
                    messages.append(HumanMessage(content=msg['content']))
                elif msg['role'] == 'ai':
                    messages.append(AIMessage(content=msg['content']))
        
        messages.append(HumanMessage(content=task))
        response = self.llm.invoke(messages)
        return response.content

In [8]:
class HistoryResearchAgent(Agent):
    def __init__(self):
        super().__init__("Clio", 
                         "History Research Specialist", 
                         ["deep knowledge of historical events","understanding of historical contexts", "identifying historical trends"]
                        )
class DataAnalysisAgent(Agent):
    def __init__(self):
        super().__init__("Data", 
                         "Data Analysis Expert", 
                         ["interpreting numerical data", "statistical analysis", "data visualization description"]
                        )

### Functions for collaboration

In [9]:
def research_historical_context(history_agent, task: str, context: list) -> list:
    print("🏛️ History Agent: Researching historical context...")
    history_task = f"Provide relevant historical context and information for the following task: {task}"
    history_result = history_agent.process(history_task)
    context.append({"role": "ai", "content": f"History Agent: {history_result}"})
    print(f"📜 Historical context provided: {history_result[:100]}...\n")
    return context

In [10]:
def identify_data_needs(data_agent, task: str, context: list) -> list:
    print("📊 Data Agent: Identifying data needs based on historical context...")
    historical_context = context[-1]["content"]
    data_need_task = f"Based on the historical context, what specific data or statistical information would be helpful to answer the original question? Historical context: {historical_context}"
    data_need_result = data_agent.process(data_need_task, context)
    context.append({"role": "ai", "content": f"Data Agent: {data_need_result}"})
    print(f"🔍 Data needs identified: {data_need_result[:100]}...\n")
    return context

In [11]:
def provide_historical_data(history_agent, task: str, context: list) -> list:
    print("🏛️ History Agent: Providing relevant historical data...")
    data_needs = context[-1]["content"]
    data_provision_task = f"Based on the data needs identified, provide relevant historical data or statistics. Data needs: {data_needs}"
    data_provision_result = history_agent.process(data_provision_task, context)
    context.append({"role": "ai", "content": f"History Agent: {data_provision_result}"})
    print(f"📊 Historical data provided: {data_provision_result[:100]}...\n")
    return context

In [15]:
def analyze_data(data_agent, task: str, context: list) -> list:
    print("📈 Data Agent: Analyzing historical data...")
    historical_data = context[-1]["content"]
    analysis_task = f"Analyze the historical data provided and describe any trends or insights relevant to the original task. Historical data: {historical_data}"
    analysis_result = data_agent.process(analysis_task, context)
    context.append({"role": "ai", "content": f"Data Agent: {analysis_result}"})
    print(f"💡 Data analysis results: {analysis_result[:100]}...\n")
    return context

In [16]:
def synthesize_final_answer(history_agent, task: str, context: list) -> str:
    print("🏛️ History Agent: Synthesizing final answer...")
    synthesis_task = "Based on all the historical context, data, and analysis, provide a comprehensive answer to the original task."
    final_result = history_agent.process(synthesis_task, context)
    return final_result

## General class

In [17]:
class HistoryDataCollaborationSystem:
    def __init__(self):
        self.history_agent = Agent("Clio", "History Research Specialist", ["deep knowledge of historical events", "understanding of historical contexts", "identifying historical trends"])
        self.data_agent = Agent("Data", "Data Analysis Expert", ["interpreting numerical data", "statistical analysis", "data visualization description"])
    
    def solve (self, task: str, timeout: int = 300) -> str:
        print(f"\n👥 Starting collaboration to solve: {task}\n")
        
        start_time = time.time()
        context = []
        
        steps = [
            (research_historical_context, self.history_agent),
            (identify_data_needs, self.data_agent),
            (provide_historical_data, self.history_agent),
            (analyze_data, self.data_agent),
            (synthesize_final_answer, self.history_agent)
        ]
        
        for step_func, agent in steps:
            if time.time() - start_time > timeout:
                return "Operation timed out. The process took too long to complete."
            try:
               result = step_func(agent, task, context)
               if isinstance(result, str):
                   return result # final answer
               context = result
            except Exception as e:
                return f"Error: {str(e)}"
            
        print("\n✅ Collaboration complete. Final answer synthesized.\n")
        return context[-1]["content"]

### Example

In [18]:
# Create an instance of the collaboration system
collaboration_system = HistoryDataCollaborationSystem()

# Define a complex historical question that requires both historical knowledge and data analysis
question = "How did urbanization rates in Europe compare to those in North America during the Industrial Revolution, and what were the main factors influencing these trends?"

# Solve the question using the collaboration system
result = collaboration_system.solve(question)

# Print the result
print(result)


👥 Starting collaboration to solve: How did urbanization rates in Europe compare to those in North America during the Industrial Revolution, and what were the main factors influencing these trends?

🏛️ History Agent: Researching historical context...
📜 Historical context provided: As a History Research Specialist, I can provide some context and information regarding urbanization ...

📊 Data Agent: Identifying data needs based on historical context...
🔍 Data needs identified: To analyze and compare urbanization rates in Europe and North America during the Industrial Revoluti...

🏛️ History Agent: Providing relevant historical data...
📊 Historical data provided: I can provide some historical data relevant to urbanization during the Industrial Revolution, but co...

📈 Data Agent: Analyzing historical data...
💡 Data analysis results: Data Agent: Analyzing the provided historical data reveals several key trends and insights regarding...

🏛️ History Agent: Synthesizing final answer...
Histor